In [1]:
import pandas as pd
import numpy as np

Confirm error has an issue logged for it:
https://github.com/pandas-dev/pandas/issues/23164

Diving deeper: https://github.com/pandas-dev/pandas/issues/16276

-------

Sounds like the making the interval slightly lower was a patch fit b/c IntervalIndex requires all bins closed on the same side.
- `(3, interval[int64]): [[0, 3] < (3, 6] < (6, 8]]` is not valid

-----
Issue below: The bounds on the interval aren't what I expect them to be. They are 0.999 vs 1.0.

They made the bottom interval slightly smaller to compensate for the fact that you can't have differently closed bins for IntervalIndex.
- Example:`(3, interval[int64]): [[0, 3] < (3, 6] < (6, 8]]` is not valid, but it's what I want.
- What I can get `[(-0.001, 3.0] < (3.0, 6.0] < (6.0, 8.0]]`
**NOTE the `[` versus the `(`**


#### "Solution"/ Hack
Just pass the labels I want used instead of using the actual IntervalIndex

In [54]:
array = [1,2,3,4,5,6,7,8]

In [58]:
pd.cut(x=array, 
       bins=[0,3,6,8], 
       include_lowest=True, 
      )


[(-0.001, 3.0], (-0.001, 3.0], (-0.001, 3.0], (3.0, 6.0], (3.0, 6.0], (3.0, 6.0], (6.0, 8.0], (6.0, 8.0], NaN]
Categories (3, interval[float64]): [(-0.001, 3.0] < (3.0, 6.0] < (6.0, 8.0]]

In [59]:
pd.cut(x=array, bins=2, include_lowest=True)


[(0.991, 5.0], (0.991, 5.0], (0.991, 5.0], (0.991, 5.0], (0.991, 5.0], (5.0, 9.0], (5.0, 9.0], (5.0, 9.0], (5.0, 9.0]]
Categories (2, interval[float64]): [(0.991, 5.0] < (5.0, 9.0]]

In [60]:
def construct_str_label(bins):
    if len(bins) < 2:
        raise ValueError(f'bin must have at least 2 values')
    bins = sorted(bins) 
    
    label = []
    
    low_idx = 0 
    for high_idx in range(1, len(bins)):
        if low_idx == 0:
            # First case is fully inclusive
            label.append(f'[{bins[low_idx]}, {bins[high_idx]}]')
        else:
            label.append(f'({bins[low_idx]}, {bins[high_idx]}]')
        low_idx += 1

    return label
            
    
construct_str_label([1,4,6])

['[1, 4]', '(4, 6]']

In [61]:
construct_str_label([1,3,4,5,7])

['[1, 3]', '(3, 4]', '(4, 5]', '(5, 7]']

### Try making an intervalIndex with multiple bracket types

In [62]:
pd.interval_range(start=0, end=5)

IntervalIndex([(0, 1], (1, 2], (2, 3], (3, 4], (4, 5]],
              closed='right',
              dtype='interval[int64]')

In [67]:
pd.IntervalIndex.from_arrays([1,4,7], [4,7,9])

IntervalIndex([(1, 4], (4, 7], (7, 9]],
              closed='right',
              dtype='interval[int64]')